# Create lithogenic and biogenic particle fields for Pb scavenging

BLING ends in April, 2019

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from mpl_toolkits.basemap import Basemap
import netCDF4 as nc
import xarray as xr
import cmocean
from scipy.spatial import Delaunay
from scipy import ndimage as nd
from scipy.interpolate import LinearNDInterpolator

%matplotlib inline

#### Parameters / settings:

In [2]:
land_color = '#a9a7a2'

# domain dimensions:
jmin, jmax = 159, 799
imin, imax = 1139, 2179
isize = imax - imin
jsize = jmax - jmin

#### Load files

In [3]:
mesh         = xr.open_dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mask_Pb-20220317.nc')
mesh_lon     = mesh['nav_lon'].values
mesh_lat     = mesh['nav_lat'].values
mesh_bathy   = mesh['tmask'][0,:,:,:].values
bathy_masked = np.ma.masked_where((mesh_bathy> 0.1), mesh_bathy)
ocean_masked = np.ma.masked_where((mesh_bathy<0.1), mesh_bathy)
depths       = mesh['nav_lev'].values

In [4]:
# BLING:
folder_EPM101 = '/data/brogalla/ANHA4/BLING-EPM101/'
dset          = xr.open_dataset(f'/ocean/brogalla/GEOTRACES/BLING-EPM101/ANHA4-EPM101_y2002m08d13_gridB.nc')
ANHA4_lons    = dset['nav_lon'].values
ANHA4_lats    = dset['nav_lat'].values
ANHA4_depths  = dset['deptht'].values

In [5]:
# Mn for surface particle field:
dset_ANHA12 = xr.open_dataset('/data/brogalla/run_storage/Mn-extended-domain-202210/oMn_y2002m01.nc')
ANHA12_lons = mesh_lon[imin:imax,jmin:jmax]
ANHA12_lats = mesh_lat[imin:imax,jmin:jmax]

#### Functions:

In [13]:
def save_file(folder, litho_array, bio_array, year, month):   
    
    file_write = xr.Dataset(
        {'lithogenic' : (("deptht","y","x"), litho_array),
         'biogenic' : (("deptht","y","x"), bio_array)}, 
        coords = {
            "time_counter": np.zeros(1),
            "deptht": depths,
            "y": np.zeros(mesh_lat.shape[0]),
            "x": np.zeros(mesh_lon.shape[1])},
    )
    file_write.to_netcdf(f'{folder}Pb_scavenging_y{year}m{month:02}.nc', \
                         unlimited_dims='time_counter')

    return

#### Calculations

In [12]:
folder = '/ocean/brogalla/GEOTRACES/data/Pb-tuning-202210/new-bio-proxy/biomass-nonlinear-2-3/'
save_folder = '/ocean/brogalla/GEOTRACES/data/Pb-tuning-202210/new-bio-proxy/biomass-nonlinear-2-3-capped/'

In [14]:
for month in range(1,13):
    f = xr.open_dataset(f'{folder}Pb_scavenging_y2002m{i:02}.nc')
    litho = f['lithogenic'].values
    bio   = f['biogenic'].values
    
    particle_field = np.add(litho,bio)
    particle_field[particle_field > 0.075] = 0.075
    
    litho_capped = litho
    bio_capped   = bio
    bio_capped[particle_field > 0.075] = 0.075
    
    save_file(f'{save_folder}', litho_capped, bio_capped, 2002, month)

In [10]:
f

<xarray.Dataset>
Dimensions:       (deptht: 50, time_counter: 1, x: 1632, y: 2400)
Coordinates:
  * time_counter  (time_counter) float64 0.0
  * deptht        (deptht) float32 0.494 1.541 2.646 ... 5.275e+03 5.728e+03
  * y             (y) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
  * x             (x) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
Data variables:
    lithogenic    (deptht, y, x) float64 ...
    biogenic      (deptht, y, x) float64 ...